In [3]:
import numpy as np
import qutip as qt
import importlib

# --- Handle both QuTiP 4.x and 5.x module layouts ---
if importlib.util.find_spec("qutip.superop_reps"):
    from qutip.superop_reps import to_super, to_choi
else:
    from qutip.core.superop_reps import to_super, to_choi

# Define a simple test channel — the identity channel
I = qt.qeye(2)
Phi = qt.spre(I)           # superoperator form (identity map)
J_qutip = to_choi(Phi)     # QuTiP's Choi matrix (4x4)
J = J_qutip.full()         # NumPy array

# Random test state
rho = qt.rand_dm(2).full()

# --- Our two candidate definitions ---
def phi_from_choi_ij(J, rho):
    """Φ(ρ) = Σ ρ[i,j] * J_block[i,j]"""
    def J_block(i, j):
        return J[2*i:2*(i+1), 2*j:2*(j+1)]
    return sum(rho[i, j] * J_block(i, j) for i in range(2) for j in range(2))

def phi_from_choi_ji(J, rho):
    """Φ(ρ) = Σ ρ[j,i] * J_block[i,j]"""
    def J_block(i, j):
        return J[2*i:2*(i+1), 2*j:2*(j+1)]
    return sum(rho[j, i] * J_block(i, j) for i in range(2) for j in range(2))

# --- Compare against QuTiP’s ground truth ---
Phi_super = to_super(Phi)
rho_vec = qt.operator_to_vector(qt.Qobj(rho))
rho_vec_out = Phi_super * rho_vec
rho_qt_out = qt.vector_to_operator(rho_vec_out).full()

# --- Results ---
print("QuTiP Φ(ρ):\n", np.round(rho_qt_out, 4))
print("Difference Φ_ij(ρ):\n", np.round(rho_qt_out, 4) - np.round(phi_from_choi_ij(J, rho), 4))
print("Difference Φ_ji(ρ):\n", np.round(rho_qt_out, 4) - np.round(phi_from_choi_ji(J, rho), 4))




QuTiP Φ(ρ):
 [[ 0.4884+0.j     -0.1877-0.4331j]
 [-0.1877+0.4331j  0.5116-0.j    ]]
Difference Φ_ij(ρ):
 [[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
Difference Φ_ji(ρ):
 [[0.+0.j     0.-0.8662j]
 [0.+0.8662j 0.+0.j    ]]
